[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana9/1_1_Delta_Hedging.ipynb)

# Introducción

Esta clase veremos que se puede hacer para reducir riesgos desde un puntos de vista distinto.

Tomaremos 3 activos, despues veremos 2 "Griegos" que son medidas utilizadas para ver tendencias y movimientos del activo (son derivadas del proceso de Black-Scholes que modela el precio de la opción de un activo financiero).

Sabemos que la derivada se puede aproximar al tomar el cociente de dos valores: la diferencia entre la evaluacion de la función en ambos puntos dividido por la diferencia entre los puntos, es decir:

$$\dfrac{dC}{dS} \approx \frac{C(X) - C(Y)}{(X) - (Y)}$$

$$\frac{C(51) - C(50)}{(51) - (50)} = \dfrac{100-110}{1} = \dfrac{-10}{1}$$

Hay varios Griegos, hoy hablamos de Delta, pero hay otros que son utiles para diferentes cosas, vean mas en el siguiente [enlace](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#The_Options_Greeks)



Utilizaremos los griegos que son:

- Delta
- Gamma

Ambos son derivadas del precio de la opción con respecto al cambio de precio del activo subyacente, una de primer orden y otra de segundo orden.

# Variables principales

Estas variables deben ser actualizadas diariamente si se quiere una buena predicción

In [ ]:
# asset_price : precio del activo subyacente
# sigma       : volatilidad implicita para insertar en el modelo BS
# dt          : tiempo de expiración
# rf          : tasa libre de riesgo (bonos del tesoro o cetes, depende el país)
# nContract1  : número de contratos
# K1          : precio de strike de opción 1
# K2          : precio de strike de opción 2
# K3          : precio de strike de opción 3

# El subyacente
asset_price = 543

# input de Black-Scholes
sigma = 0.53
dt = 30/365
rf = .015

# opción 1
nContract1 = -1000
K1 = 545

# opción 2
K2 = 540

# opción 3
K3 = 555

# Dependencias

In [ ]:
import math
from scipy.stats import norm
import numpy as np

# Ejercicio 1:

¿Porque importar "norm" de "scipy"?

Recordemos todo lo necesario para calcular las opciones en el siguiente [enlace](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#Black%E2%80%93Scholes_formula)

In [ ]:
# Define una opción de tipo Call Europeo
class EuropeanCall:
    # Call delta
    def call_delta(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        return z1

    # Call gamma
    def call_gamma(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z2 = z1 / (asset_price * asset_volatility * math.sqrt(time_to_expiration))
        return z2

    def call_price(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z1 = z1 * asset_price
        x2 = math.log(asset_price / (b * strike_price)) - .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x2 = x2 / (asset_volatility * (time_to_expiration ** .5))
        z2 = norm.cdf(x2)
        z2 = b * strike_price * z2
        return z1 - z2

    def __init__(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.call_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.call_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.gamma = self.call_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

In [ ]:
# Define una opción de tipo Put Europeo
class EuropeanPut:
    # Put delta
    def put_delta(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        return z1 - 1

    # Put gamma
    def put_gamma(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z2 = z1 / (asset_price * asset_volatility * math.sqrt(time_to_expiration))
        return z2

    def put_price(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log((b * strike_price) / asset_price) + .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z1 = b * strike_price * z1
        x2 = math.log((b * strike_price) / asset_price) - .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x2 = x2 / (asset_volatility * (time_to_expiration ** .5))
        z2 = norm.cdf(x2)
        z2 = asset_price * z2
        return z1 - z2

    def __init__(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.put_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.call_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.gamma = self.put_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

In [ ]:
# Posición con una opción
option1 = EuropeanCall(asset_price=asset_price, asset_volatility=sigma, strike_price=K1, time_to_expiration=dt,risk_free_rate=rf)

# Valor teorico de la posición:
print('Theoretical Initial Portfolio value: ', str(option1.price * abs(nContract1)))

# greeks
print('Initial Portfolio Greeks:\n '
      'Delta: {}\n '
      'Gamma: {}'.format(option1.delta * nContract1, option1.gamma * nContract1))



Theoretical Initial Portfolio value:  32264.05329034736
Initial Portfolio Greeks:
 Delta: -523.8788365375873
 Gamma: -6.3495209433350475


In [ ]:
option1

# Calcular call y griegos de las otras 2 opciones

In [ ]:
# Precio de option2, option3 y los griegos de estos
option2 = EuropeanCall(asset_price=asset_price, asset_volatility=sigma, strike_price=K2, time_to_expiration=dt, risk_free_rate=rf)

option3 = EuropeanCall(asset_price=asset_price, asset_volatility=sigma, strike_price=K3, time_to_expiration=dt, risk_free_rate=rf)

In [ ]:
print('Initial Portfolio Greeks:\n '
      'Delta: {}\n '
      'Gamma: {}'.format(option2.delta, option2.gamma))

Initial Portfolio Greeks:
 Delta: 0.5479756607145875
 Gamma: 0.006641579486472526


In [ ]:
print('Initial Portfolio Greeks:\n '
      'Delta: {}\n '
      'Gamma: {}'.format(option3.delta, option3.gamma))

Initial Portfolio Greeks:
 Delta: 0.47616874797224407
 Gamma: 0.005771264702719988


# Neutralización de ambos griegos

Explicación matemática de lo que sucederá adelante

In [ ]:
portfolio_greeks = [[option1.gamma * abs(nContract1)], [option1.delta * abs(nContract1)]]
portfolio_greeks

[[6.3495209433350475], [523.8788365375873]]

Valores de los griegos de la opción 1 (lo que está en nuestro portafolio hasta el momento)

$ \begin{bmatrix} gamma \\ delta \end{bmatrix} = \begin{bmatrix} -6.3495... \\ -523.8788 \end{bmatrix} $

In [ ]:
greeks = np.array([[option2.gamma, option3.gamma], [option2.delta, option3.delta]])
greeks

array([[0.00664158, 0.00577126],
       [0.54797566, 0.47616875]])

In [ ]:
print(f'Gamma de opción 2: {np.round(option2.gamma,4)}\n'
      f'Delta de opción 2: {np.round(option2.delta,4)}\n'
      f'Gamma de opción 3: {np.round(option3.gamma,4)}\n'
      f'Delta de opción 3: {np.round(option3.delta,4)}')

Gamma de opción 2: 0.0066
Delta de opción 2: 0.548
Gamma de opción 3: 0.0058
Delta de opción 3: 0.4762


Queremos encontrar la posicion que tendríamos respecto a las opciones 2 y 3, dados su griegos.

$\begin{bmatrix} 0.00605 & 0.00577 \\ 0.49991 & 0.47616 \end{bmatrix} \begin{bmatrix} w_2 \\ w_3 \end{bmatrix} = \begin{bmatrix} 6.35 \\ 523.88 \end{bmatrix}$


$$w = A^{-1} \cdot A \cdot w = A^{-1} \cdot P  $$

# Ejercicio 2:
¿Qué podemos hacer para resolverlo?

In [ ]:
# Neutralización de griegos -- delta y gamma
inv = np.linalg.inv(np.round(greeks, 2))  # Es recomendable hacer el redondeo para evitar problemas con la inversión de matrices

# Posiciones en las opciones 2 y 3 para neutralizar delta y gamma
w = np.dot(inv, portfolio_greeks)

Posiciones que se deben tomar respecto a las opciones 2 y 3

In [ ]:
w

array([[ 3130.02616082],
       [-2495.07406649]])

In [ ]:
print(f'Posiciones Finales: \n'
        f'Opción 1: {nContract1} \n'
        f'Opción 2: {w[0][0]} \n'
        f'Opción 3: {w[1][0]}')

Posiciones Finales: 
Opción 1: -1000 
Opción 2: 3130.0261608214973 
Opción 3: -2495.074066487992


# Ejercicio 3:

Para algún activo de yahoo finance, apliquen el algoritmo.

Tickers sugeridos:
- GOOGL - Google
- APPL - Apple
- NVDA - Nvidia
- IBM
- INTC - Intel corporation
- SHEL - Petrolera
- NUE - Nucor Corporation (acerera estadounidense)

Tasa libre de riesgo:
- ^IRX - Bonos del tesoro a 3 meses (recuerden que hay que divirilo entre 100, hoy esta alrededor del 5%)


In [ ]:
# Espacio para ejercicio